In [ ]:
import os
import heartpy as hp
import numpy as np
import pandas as pd
from scipy import signal, integrate
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from wfdb import rdheader, rdrecord
from neurokit2.ppg import ppg_findpeaks

pd.options.plotting.backend = 'plotly'
os.chdir('/home/camhpj/database_tools/')

def download(path):
    response = os.system(f'wget -q -r -np {path}')
    return response

def get_sigs(folder, seg):
    path = 'physionet.org/files/mimic3wdb/1.0/' + folder + seg
    response = download(path + '.hea')
    response = download(path + '.dat')
    if response == 0:
        rec = rdrecord(path)
        signals = rec.sig_name
        pleth = rec.p_signal[:, signals.index('PLETH')].astype(np.float64)
        abp = rec.p_signal[:, signals.index('ABP')].astype(np.float64)
        return pleth, abp
    return None, None

def time_similarity(x, y):
    x_bar = np.mean(x)
    y_bar = np.mean(y)
    y_temp = y - y_bar
    top = np.sum( ((x - x_bar) * y_temp) )
    bot = np.sqrt( np.sum( ((x - y_bar) ** 2) ) * np.sum( ((y_temp) ** 2) ) )
    coef = top / bot
    return coef

def hr_conflict(x, y, fs=125):
    t = len(x) / fs  # length of data in sec
    x_peaks = len(ppg_findpeaks(x, fs)['PPG_Peaks'])
    y_peaks = len(ppg_findpeaks(y, fs)['PPG_Peaks'])
    print(x_peaks, y_peaks)
    if np.abs((x_peaks / t) - (y_peaks / t)) > 10:
        return True
    return False

def bandpass(x, band=(0.5, 8), fs=125):
    low, high = band[0], band[1]
    btr = signal.butter(4, [low, high], btype='bandpass', output='sos', fs=fs)
    x = signal.sosfiltfilt(btr, x, padtype=None)
    return x

def calculate_snr(x, band, fs=125):
    low, high = band[0], band[1]
    win = fs * 5

    # Signal power
    freqs, psd = signal.welch(x, fs, nperseg=win)
    freq_res = freqs[1] - freqs[0]
    idx_delta = np.logical_and(freqs >= low, freqs <= high)
    p_sig = integrate.simps(psd[idx_delta], dx=freq_res)

    freqs, psd = signal.welch(pleth, fs, nperseg=win)
    freq_res = freqs[1] - freqs[0]
    idx_delta = freqs < low
    p1 = integrate.simps(psd[idx_delta], dx=freq_res)
    idx_delta = freqs > high
    p2 = integrate.simps(psd[idx_delta], dx=freq_res)
    p_noise = p1 + p2

    snr = 10 * np.log10(p_sig / p_noise)
    return snr

pleth, abp = get_sigs('30/3000480/', '3000480_0018')
pleth[np.isnan(pleth)] = 0
abp[np.isnan(abp)] = 0

In [ ]:
i = 1024

aligned = []
for j in range(1, 100):
    x = pleth[i*j:i*(j+1)]
    y = abp[i*j:i*(j+1)]

    sim = -1
    sim_last = -2
    offset = 0
    while sim > sim_last:
        sim_last = sim
        x = pleth[i*j-offset:i*(j+1)-offset]
        sim = np.sum(x * y)
        offset += 1
    aligned.append([x, y])

In [ ]:
i = 0
x, y = aligned[i][0], aligned[i][1]

scaler = MinMaxScaler(feature_range=(60, 120))
x = scaler.fit_transform(x.reshape(-1, 1)).reshape(-1,)

df = pd.DataFrame(dict(pleth=x,
                        abp=y))
df.plot()

In [ ]:
i = 1024
j = 3
offset = 0
x = pleth[i*j-offset:i*(j+1)-offset]
y = abp[i*j:i*(j+1)]

In [ ]:
# scaler = MinMaxScaler(feature_range=(np.min(y), np.max(y)))
# x = scaler.fit_transform(x.reshape(-1, 1)).reshape(-1,)

In [ ]:
time_similarity(pleth, abp)

In [ ]:
pleth = bandpass(pleth)
calculate_snr(pleth, (0.5, 8), 125)

In [ ]:
calculate_snr(abp, (0.5, 2.5), 125)

In [ ]:
for i in [2, 4, 8, 16, 32, 64, 128]:
    print( (100 / i) + 4)

In [ ]:
def parabolic(f, x):
    xv = 1/2. * (f[x-1] - f[x+1]) / (f[x-1] - 2 * f[x] + f[x+1]) + x
    yv = f[x] - 1/4. * (f[x-1] - f[x+1]) * (xv - x)
    return (xv, yv)

def freq_from_crossings(sig, fs):
    """
    Estimate frequency by counting zero crossings
    """
    # Find all indices right before a rising-edge zero crossing
    indices = np.nonzero((sig[1:] >= 0) & (sig[:-1] < 0))[0]

    # Naive (Measures 1000.185 Hz for 1000 Hz, for instance)
    # crossings = indices

    # More accurate, using linear interpolation to find intersample
    # zero-crossings (Measures 1000.000129 Hz for 1000 Hz, for instance)
    crossings = [i - sig[i] / (sig[i+1] - sig[i]) for i in indices]

    # Some other interpolation based on neighboring points might be better.
    # Spline, cubic, whatever

    return fs / np.mean(np.diff(crossings))

def freq_from_autocorr(sig, fs):
    """
    Estimate frequency using autocorrelation
    """
    # Calculate autocorrelation and throw away the negative lags
    corr = np.correlate(sig, sig, mode='full')
    corr = corr[len(corr)//2:]

    # Find the first low point
    d = np.diff(corr)
    start = np.nonzero(d > 0)[0][0]

    # Find the next peak after the low point (other than 0 lag).  This bit is
    # not reliable for long signals, due to the desired peak occurring between
    # samples, and other peaks appearing higher.
    # Should use a weighting function to de-emphasize the peaks at longer lags.
    peak = np.argmax(corr[start:]) + start
    px, py = parabolic(corr, peak)

    return fs / px

def freq_from_fft(sig, fs):
    """
    Estimate frequency from peak of FFT
    """
    # Compute Fourier transform of windowed signal
    windowed = sig * signal.blackmanharris(len(sig))
    f = np.fft.rfft(windowed)

    # Find the peak and interpolate to get a more accurate peak
    i = np.argmax(abs(f))  # Just use this for less-accurate, naive version
    true_i = parabolic(np.log(abs(f)), i)[0]

    # Convert to equivalent frequency
    return fs * true_i / len(windowed)

In [ ]:
fs = 125
t = 100
x = np.linspace(-np.pi, t * np.pi, 1000)
x = np.sin(x)
pd.Series(x).plot()

freq_from_fft(x, 125)

In [ ]:
freq_from_fft(abp, 125)

In [ ]:
freq_from_autocorr(abp, 125)

In [ ]:
abp = abp - np.mean(abp)
freq_from_crossings(abp, 125)

In [ ]:
60 * 1.7

In [ ]:
pd.Series(abp[0:1250]).plot()

In [ ]:
pleth, _ = hp.load_exampledata(0)
pleth = pleth + np.random.normal(0,100,2483)
pd.Series(pleth).plot()

In [ ]:
from scipy import signal, integrate

fs = 100
win = fs * 5
low, high = 0.3, 30

btr = signal.butter(4, [low, high], btype='bandpass', output='sos', fs=fs)
pleth = signal.sosfiltfilt(btr, pleth, padtype=None)

# Signal power
freqs, psd = signal.welch(pleth, fs, nperseg=win)
freq_res = freqs[1] - freqs[0]

idx_delta = np.logical_and(freqs >= 0.5, freqs <= 8)
p_sig = integrate.simps(psd[idx_delta], dx=freq_res)

# Signal noise
freqs, psd = signal.welch(pleth, fs, nperseg=win)
freq_res = freqs[1] - freqs[0]

idx_delta = freqs < 0.5
p1 = integrate.simps(psd[idx_delta], dx=freq_res)
idx_delta = freqs > 8
p2 = integrate.simps(psd[idx_delta], dx=freq_res)
p_noise = p1 + p2

snr = 10 * np.log10(p_sig / p_noise)
snr